In [ ]:
import numpy as np
import mxnet as mx
from mxnet import nd
from mxnet import gluon
from mxnet import autograd
from mxnet.gluon import nn

In [ ]:
# Basic use of Blocks
# 1. Define a class that encapsulates the parameters and the forward pass
# 2. Define an optimizer
# 3. Compute the loss function value
# 4. Use auto differentiation to compute gradients
# 5. Update parameters
x = nd.array([[0.0, 0.0],
              [0.0, 1.0],
              [1.0, 0.0],
              [1.0, 1.0]])
y = nd.array([[0.0],
              [1.0],
              [1.0],
              [0.0]])

n = x.shape[0]
p = x.shape[1]
d = y.shape[1]
r = 10

class MyModel(nn.Block):
    def __init__(self, in_dim, out_dim, hidden_dim):
        super(MyModel, self).__init__()
        self.fc1 = nn.Dense(hidden_dim, in_units=in_dim)
        self.fc2 = nn.Dense(out_dim, in_units=hidden_dim)

    def forward(self, x):
        z1 = self.fc1(x)
        a1 = nd.Activation(z1, "softrelu")
        z2 = self.fc2(a1)
        phat = nd.sigmoid(z2)
        return phat

np.random.seed(123)
mx.random.seed(123)

model = MyModel(in_dim=p, out_dim=d, hidden_dim=r)
model.initialize(mx.init.Normal(sigma=0.1))
print(model, "\n")
print(model.collect_params())

In [ ]:
nepoch = 3000
learning_rate = 0.2
opt = gluon.Trainer(model.collect_params(), "sgd", {"learning_rate": learning_rate})

for i in range(nepoch):
    with autograd.record():
        phat = model(x)
        loss = nd.mean(-y * nd.log(phat) - (1.0 - y) * nd.log(1.0 - phat))

    loss.backward()
    opt.step(1)
    
    if i % 100 == 0:
        print(f"iteration {i}, loss = {loss.asscalar()}, prediction = {phat.asnumpy().squeeze()}")